In [1]:
from selenium import webdriver
from pymongo import MongoClient
from flask import Flask, render_template, jsonify

In [2]:
from selenium.webdriver.common.keys import Keys

In [3]:
import pandas as pd

In [4]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [5]:
pd.DataFrame(columns=["Caractéristiques(Chambres,Douches et Surfaces)","Emplacements","Prix"])

,"Caractéristiques(Chambres,Douches et Surfaces)",Emplacements,Prix


In [6]:
service=Service('C:/Users/hechm/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe')

In [7]:
driver=webdriver.Chrome(service=service)

In [8]:
caracteristiques_list=[]
emplacements_list=[]
prix_list=[]

In [9]:
def extraire_donnees(url_pattern, page_range):
    caracteristiques = []
    emplacements = []
    prix = []

    for i in page_range:
        url = url_pattern.format(i)
        driver.get(url)
        
        # Extraction des caractéristiques
        caracteristiques.extend([c.text for c in driver.find_elements(By.CLASS_NAME, 'li-list-item-info-span')])
        
        # Extraction des emplacements à partir des balises <p>
        emplacements.extend([p.text for p in driver.find_elements(By.TAG_NAME, 'p') if p.text.strip() != ""])
        
        # Extraction des prix
        prix.extend([p.text for p in driver.find_elements(By.CLASS_NAME, 'item-box-price')])

    return caracteristiques, emplacements, prix

In [10]:
page_range = range(1, 3)
appartements_url = "https://www.menzili.tn/immo/appartement-tunisie?l=0&page={}&tri=1"
maisons_url = "https://www.menzili.tn/immo/maison-tunisie?l=0&page={}&tri=1"
terrains_url = "https://www.menzili.tn/immo/terrain-tunisie?l=0&page={}&tri=1"

In [11]:
caracteristiques_appartements, emplacements_appartements, prix_appartements = extraire_donnees(appartements_url, page_range)
caracteristiques_maisons, emplacements_maisons, prix_maisons = extraire_donnees(maisons_url, page_range)
caracteristiques_terrains, emplacements_terrains, prix_terrains = extraire_donnees(terrains_url, page_range)

In [12]:
for a in caracteristiques_appartements:
    caracteristiques_list.append(a.split())
for m in caracteristiques_maisons:
    caracteristiques_list.append(m.split())
for t in caracteristiques_terrains:
    caracteristiques_list.append(t.split())

for ea in emplacements_appartements:
    emplacements_list.append(ea.strip())
for em in emplacements_maisons:
    emplacements_list.append(em.strip())
for et in emplacements_terrains:
    emplacements_list.append(et.strip())

for pa in prix_appartements:
    prix_list.append(pa.strip())
for pm in prix_maisons:
    prix_list.append(pm.strip())
for pt in prix_terrains:
    prix_list.append(pt.strip())

data_tuples=list(zip(caracteristiques_list,emplacements_list,prix_list))
pd.DataFrame(data_tuples,columns=["Caractéristiques(Chambres,Douches et Surfaces)","Emplacements","Prix"])

,"Caractéristiques(Chambres,Douches et Surfaces)",Emplacements,Prix
0,"[2, Chambres, 1, Salle, de, bain, 5, Piéces, (...",En poursuivant votre navigation sur notre site...,175 DT
1,"[2, Chambres, 1, Salle, de, bain, 5, Piéces, (...","Djerba-Houmt Souk, Médenine",140 DT
2,"[2, Chambres, 1, Salle, de, bain, 5, Piéces, (...","Djerba-Houmt Souk, Médenine",140 DT
3,"[2, Chambres, 1, Salle, de, bain, 5, Piéces, (...","Djerba-Houmt Souk, Médenine",175 DT
4,"[2, Chambres, 1, Salle, de, bain, 5, Piéces, (...","Djerba-Houmt Souk, Médenine",210 DT
...,...,...,...
115,"[21000m², Surf, terrain]",Tunis centre ville,1 000 000 DT
116,"[480000m², Surf, terrain]",Sousse,105 000 DT
117,[],Sfax,79 000 DT
118,"[250m², Surf, terrain]",Monastir,2 600 000 DT


In [13]:
def inserer_annonces():
    client=MongoClient('mongodb://localhost:27017/')
    annonces=[{"Caracteristiques(Chambres,Douches,Surface)":caracteristique,"Emplacements":emplacement,"Prix":prix} for caracteristique,emplacement,prix in zip(caracteristiques_list,emplacements_list,prix_list)]
    with client:
        db=client.immobilieres
        db.annonces.insert_many(annonces)

In [14]:
def afficher_annonces():
    client=MongoClient('mongodb://localhost:27017/')
    with client:
        db=client.immobilieres
        annonces=db.annonces.find()
        annonces.next()
        annonces.next()
        annonces.next()
        annonces.next()
        annonces.next()
        annonces.next()
        annonces.rewind()
        print(list(annonces))

In [15]:
def compter_annonces():
    client=MongoClient('mongodb://localhost:27017/')
    with client:
        db=client.immobilieres
        print(f"Il existe {db.annonces.count_documents({})} annonces immobilières")

In [16]:
app=Flask(__name__)
@app.route('/scrape')
def scrape():
    try:
        # Modèles d'URL et plages de pages à scraper
        urls = [
            "https://www.menzili.tn/immo/appartement-tunisie?l=0&page={}",
            "https://www.menzili.tn/immo/maison-tunisie?l=0&page={}",
            "https://www.menzili.tn/immo/terrain-tunisie?l=0&page={}"
        ]
        page_range = range(1, 3)

        for url_pattern in urls:
            caracteristiques, emplacements, prix = extraire_donnees(url_pattern, page_range)

            # Insérer les données dans MongoDB
            inserer_annonces()

        return jsonify({"message": "Scraping complet!"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


# Route pour afficher les données
@app.route('/data')
def data():
    try:
        annonces = afficher_annonces()
        return render_template('data.twig', annonces=annonces)
    except Exception as e:
        return jsonify({"error": str(e)}), 500


# Route pour les statistiques
@app.route('/stats')
def stats():
    try:
        count = compter_documents()
        return jsonify({"total_listings": count})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\hechm\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [17]:
driver.quit()